In [124]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [125]:
# read data
DATA_FOLDER_PATH = os.getcwd() + '/../data/KS VERI/'
FILE_NAME = 'KS10_FINAL_DATA.csv'
PATH = DATA_FOLDER_PATH + FILE_NAME
FINAL_DF = pd.read_csv(PATH, header=0)
FINAL_DF['Timestamp'] = pd.to_datetime(FINAL_DF['Timestamp'])

In [126]:
## PUE detect outliers
outliers = FINAL_DF[(FINAL_DF['PUE'] > 2.5) | (FINAL_DF['PUE'] < 1)]

## REPLACE OUTLIERS WITH THE MEAN OF THE ADJACENT VALUES
for i in outliers.index:
    if i == 0:
        FINAL_DF.loc[i, 'PUE'] = FINAL_DF.loc[i + 1, 'PUE']
    elif i == len(FINAL_DF) - 1:
        FINAL_DF.loc[i, 'PUE'] = FINAL_DF.loc[i - 1, 'PUE']
    else:
        FINAL_DF.loc[i, 'PUE'] = (FINAL_DF.loc[i - 1, 'PUE'] + FINAL_DF.loc[i + 1, 'PUE']) / 2


In [127]:
DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
HOURS = ['hour_' + str(hour) for hour in range(24)]
MONTHS = ['month_' + str(month) for month in range(4, 11)]
column_names = ['Timestamp', 'PUE',
                    
                    'I_KOMP1_HIZ', 
                    #'I_KOMP1_SAAT',
                    #'I_KOMP2_HIZ', 
                    #'I_KOMP2_SAAT',
                    #'I_NEM_SET','I_SIC_SET', 
                    
                    'II_KOMP1_HIZ', 
                    #'II_KOMP1_SAAT',
                    #'II_KOMP2_HIZ',
                    #'II_KOMP2_SAAT',
                    #'II_NEM_SET','II_SIC_SET', 
                    
                    'III_KOMP1_HIZ', 
                    #'III_KOMP1_SAAT',
                    #'III_KOMP2_HIZ', 
                    #'III_KOMP2_SAAT',
                    #'III_NEM_SET','III_SIC_SET', 
                    
                    'IV_KOMP1_HIZ', 
                    #'IV_KOMP1_SAAT',
                    #'IV_KOMP2_HIZ', 
                    #'IV_KOMP2_SAAT',
                    #'IV_NEM_SET',
                    
                    'CH1_CIKIS_SIC', 'CH1_GIRIS_SIC',
                    'CH2_CIKIS_SIC', 'CH2_GIRIS_SIC', 
                    'CH3_CIKIS_SIC', 'CH3_GIRIS_SIC',
                    
                    'SENSOR_I_TEMP', 'SENSOR_II_TEMP', 
                    'OUTLET_TEMP', 'OUTLET_HUMIDITY'] + DAYS + HOURS + MONTHS

In [128]:
FINAL_DF = FINAL_DF.loc[:, column_names]

In [129]:
# add extra columns

#FINAL_DF["PUE_lag_" + str(1)] = FINAL_DF["PUE"].shift(1)
FINAL_DF["PUE_lag_one_day"] = FINAL_DF["PUE"].shift(24*12)
#FINAL_DF["PUE_lag_one_week"] = FINAL_DF["PUE"].shift(7*24*12)


FINAL_DF.dropna(inplace=True)
FINAL_DF.reset_index(drop=True, inplace=True)

In [130]:
train, test = train_test_split(FINAL_DF, test_size=0.2, random_state=42)

In [131]:
pue = sm.load("PUE_OLS.pkl")
coefs_pue = pue.params
sensor1 = sm.load("SENSOR_I_OLS.pkl")
coefs_sensor1 = sensor1.params
sensor2 = sm.load("SENSOR_II_OLS.pkl")
coefs_sensor2 = sensor2.params


In [132]:
index_cols = dict(zip(range(4), ['I_KOMP1_HIZ', 'II_KOMP1_HIZ', 'III_KOMP1_HIZ', 'IV_KOMP1_HIZ']))

In [142]:
## write optimization model with gurobi
# create a new model
for i in range(100):
    test_data = test.iloc[i]    
    M = gp.Model("ks10")
    coef = 1
    v = M.addVars(range(4), lb=0, ub=100, vtype=GRB.CONTINUOUS, name="v")

    M.setObjective(coefs_pue.I_KOMP1_HIZ * v[0] + 
                    coefs_pue.II_KOMP1_HIZ * v[1] + 
                    coefs_pue.III_KOMP1_HIZ * v[2] + 
                    coefs_pue.IV_KOMP1_HIZ * v[3] +
                    sum([coefs_pue[col] * test_data[col] for col in train.columns[6:]])
                , GRB.MINIMIZE)

    c1 = M.addConstr(
        coefs_sensor1['I_KOMP1_HIZ'] * v[0] + coefs_sensor1['II_KOMP1_HIZ'] * v[1] + coefs_sensor1['III_KOMP1_HIZ'] * v[2] + coefs_sensor1['IV_KOMP1_HIZ'] * v[3]
        + sum([coefs_sensor1[col] * test_data[col] for col in coefs_sensor1.index if "KOMP" not in col and col != "Intercept"]) + coefs_sensor1.Intercept <= np.log(24)
    )

    c2 = M.addConstr(
        coefs_sensor1['I_KOMP1_HIZ'] * v[0] + coefs_sensor1['II_KOMP1_HIZ'] * v[1] + coefs_sensor1['III_KOMP1_HIZ'] * v[2] + coefs_sensor1['IV_KOMP1_HIZ'] * v[3]
        + sum([coefs_sensor1[col] * test_data[col] for col in coefs_sensor1.index if "KOMP" not in col and col != "Intercept"]) + coefs_sensor1.Intercept >= np.log(16)
    )

    c3 = M.addConstr(
        coefs_sensor2['I_KOMP1_HIZ'] * v[0] + coefs_sensor2['II_KOMP1_HIZ'] * v[1] + coefs_sensor2['III_KOMP1_HIZ'] * v[2] + coefs_sensor2['IV_KOMP1_HIZ'] * v[3]
        + sum([coefs_sensor2[col] * test_data[col] for col in coefs_sensor2.index if "KOMP" not in col and col != "Intercept"]) + coefs_sensor1.Intercept <= np.log(24)
    )

    c4 = M.addConstr(
        coefs_sensor2['I_KOMP1_HIZ'] * v[0] + coefs_sensor2['II_KOMP1_HIZ'] * v[1] + coefs_sensor2['III_KOMP1_HIZ'] * v[2] + coefs_sensor2['IV_KOMP1_HIZ'] * v[3]
        + sum([coefs_sensor2[col] * test_data[col] for col in coefs_sensor2.index if "KOMP" not in col and col != "Intercept"]) + coefs_sensor1.Intercept >= np.log(16)
    )

    c5 = M.addConstrs(v[i] >= 0.2*FINAL_DF.loc[int(test_data.name)-1][index_cols[i]] for i in range(4))
    c6 = M.addConstrs(v[i] <= 1.7*FINAL_DF.loc[int(test_data.name)-1][index_cols[i]] for i in range(4))
    ## solve gurobi model
    M.optimize()

    ## print results
    for v_ in M.getVars():
        print('%s %g' % (v_.varName, v_.x))

    print(f'Obj: {np.exp(M.objVal)+1}')
    for i in range(4):
        print(f"Difference: {v[i].x - FINAL_DF.loc[int(test_data.name)-1][index_cols[i]]}")
    print("****************************"*3)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 4 columns and 24 nonzeros
Model fingerprint: 0x37a332e9
Coefficient statistics:
  Matrix range     [2e-06, 1e+00]
  Objective range  [2e-03, 4e-03]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-01, 2e+02]
Presolve removed 12 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.8556322e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective -1.855632221e+00
v[0] 0
v[1] 6.64
v[2] 17.72
v[3] 5
Obj: 1.1563540610860437
Difference: 0.0
Difference: -26.5600006103516
Difference: -70.87999877929687
Difference: -20.0
************************************************************************************
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm

AttributeError: Unable to retrieve attribute 'x'